In [8]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import trapz

TRAINING DATASET

In [6]:
train_images_df = pd.read_csv('train_images.csv')
train_images_df.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.341,0.342,0.343,0.344,0.345,0.346,0.347,0.348,0.349,0.350
0,0,0,0,0,0,1,0,0,0,0,...,119,114,130,76,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,22,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,33,96,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,22,...,0,0,0,0,133,167,73,0,0,0


In [7]:
train_labels_df = pd.read_csv('train_labels.csv')
train_labels_df.head()

,9
0,0
1,0
2,3
3,0
4,2


TESTING DATASET

In [4]:
test_images_df = pd.read_csv('test_images.csv')
test_images_df.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.507,0.508,0.509,0.510,0.511,0.512,0.513,0.514,0.515,0.516
0,0,0,0,0,0,0,0,0,0,0,...,2,3,0,3,174,189,67,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,164,58,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,21,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,2,0,1,1,0,0,0,...,71,12,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,145,129,62,0,0,0,0,0,0,0


In [5]:
test_labels_df = pd.read_csv('test_labels.csv')
test_labels_df.head()


,9
0,2
1,1
2,1
3,6
4,1


Binarization – Select a threshold, ‘del’, to be 127: 

All pixel values less than ‘del’ are given value 0 
All pixel values greater than or equal to ‘del’ are given value 1 
